In [38]:
import pandas as pd
import matplotlib

In [39]:
data = pd.read_csv('data/train_data/train_task_3_4.csv')
print(len(data))
data.head()

1382727


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,898,2111,280203,1,2,2
1,767,3062,55638,1,3,3
2,165,1156,386475,1,2,2
3,490,1653,997498,1,4,4
4,298,3912,578636,1,3,3


In [40]:
answer_metadata = pd.read_csv('data/metadata/answer_metadata_task_3_4.csv')
data=data.merge(answer_metadata, on="AnswerId", how="left")
data=data.groupby("QuestionId")[["Confidence","IsCorrect"]].mean()

# Create avg confidence
data.dropna(subset=["Confidence","IsCorrect"])

data

,Confidence,IsCorrect
QuestionId,,
0,66.176471,0.443457
1,NaN,0.571429
2,66.201354,0.385214
3,85.021246,0.808757
4,59.343434,0.401408
...,...,...
943,83.793687,0.566528
944,69.852941,0.142857
945,72.244733,0.422336


In [41]:

# data=data.sort_values("Confidence",ascending=False).reset_index()
data=data.sort_values("IsCorrect",ascending=True).reset_index()
data


,QuestionId,Confidence,IsCorrect
0,841,NaN,1.000000
1,847,NaN,1.000000
2,660,NaN,1.000000
3,924,93.435155,0.923497
4,422,92.968750,0.895833
...,...,...,...
943,718,67.105263,0.183891
944,425,75.000000,0.179487
945,155,79.571984,0.164776
946,931,76.018519,0.160400


In [42]:
eval_validation = pd.read_csv('data/test_data/quality_response_remapped_public.csv')
print(len(eval_validation))
eval_validation.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,0,0,909,318,2,2,2,1.0,2
1,16,16,285,609,2,2,2,2.0,2
2,48,48,132,271,1,2,2,2.0,1
3,38,38,704,127,2,2,1,2.0,2
4,33,33,16,586,1,1,2,2.0,2


In [43]:
eval_test = pd.read_csv('data/test_data/quality_response_remapped_private.csv')
print(len(eval_test))
eval_test.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,15,15,145,762,2,2,2,2.0,2
1,31,31,307,208,1,1,1,2.0,1
2,25,25,278,195,1,1,1,1.0,1
3,3,3,13,153,2,2,2,2.0,2
4,40,40,23,100,2,1,1,1.0,1


In [44]:
submission = pd.read_csv('submission/template.csv')
print(len(submission))
print(submission.head())
for idx in range(len(data)):
    tmp = data[data['QuestionId']==idx].index
    submission.loc[idx,'ranking'] = 987654321+idx if tmp.empty else tmp[0]
print(data[710:720])
submission.head()

948
   QuestionId  ranking
0           0      NaN
1           1      NaN
2           2      NaN
3           3      NaN
4           4      NaN
     QuestionId  Confidence  IsCorrect
710         632   73.611111   0.386861
711         215   67.226891   0.386747
712         467   70.833333   0.386602
713         918   64.166667   0.386243
714          84   69.040404   0.385992
715         640   54.605263   0.385928
716         903   68.338816   0.385793
717         385   63.846154   0.385743
718         869   63.611111   0.385737
719          32   90.000000   0.385714


,QuestionId,ranking
0,0,593.0
1,1,327.0
2,2,720.0
3,3,26.0
4,4,675.0


In [45]:
def order2preference(ord, QL, QR):
    preference = []
    for idx in range(len(QL)):
        rankL = ord[ord.QuestionId==QL[idx]].ranking.values[0]
        rankR = ord[ord.QuestionId==QR[idx]].ranking.values[0]
        preference.append(1 if rankL<rankR else 2)
    return preference

def evaluate(truth, submission):
    QL = list(truth.left)
    QR = list(truth.right)
    if len(QL) != len(QR):
        message = 'left and right lengths are not the same'
        exit(message)
    print(list(truth["preference"]))
    return sum(x==y for x,y in zip(truth["preference"],order2preference(submission,QL,QR)))/len(QL)

def evaluates(truth, submission):
    return [
        evaluate(truth.rename(columns={"T1_ALR":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T2_CL":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T3_GF":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T4_MQ":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T5_NS":"preference"})[["left","right","preference"]], submission)]


In [46]:
list_vali = evaluates(eval_validation,submission)
print(list(order2preference(submission,list(eval_validation.left),list(eval_validation.right))))
print(max(list_vali))
print(list_vali)

[2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1]
[2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1]
[2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1]
[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0]
[2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1]
[1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2]
0.44
[0.28, 0.32, 0.24, 0.44, 0.28]


In [47]:
list_test = evaluates(eval_test,submission)
print(max(list_test))
print(list_test)
print(list(order2preference(submission,list(eval_test.left),list(eval_test.right))))

[2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2]
[2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, nan, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0]
[2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2]
0.44
[0.4, 0.44, 0.36, 0.32, 0.4]
[1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
